In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cd drive && cd MyDrive && cd colab_test && ls

GoogleNews-vectors-negative300.bin  lecture2.ipynb  tokenize.ipynb
kakao.txt			    lecture3.ipynb
learning_1.ipynb		    lecture4.ipynb


In [4]:
nickname = '나한솔' #학습 대상 닉네임
nickname = '[' + nickname
f = open('drive/MyDrive/colab_test/kakao.txt', 'r', encoding='UTF-8')
lines = f.readlines()[5:]
df = pd.DataFrame(columns=['questions', 'answers'])

In [5]:
name = msg = '' #바로 이전 대화 저장
for line in tqdm(lines):
    #대화가 아닌 부분에 대한 예외처리
    if line.find('들어왔습니다.')>0 or line.find('나갔습니다.')>0 or line.find('---------------')>=0: continue
    if line.find('[오전')==-1 and line.find('[오후')==-1: continue
        
    try:
        last_name, last_msg = name, msg
        name, msg = line.split('] [')
        msg = msg.split('] ',1)[1].strip()
        #링크가 들어간 부분은 제외
        if name==nickname and last_name!=nickname and last_msg.find('http')==-1 and msg.find('http')==-1:
            df = df.append({'questions':last_msg, 'answers':msg}, ignore_index=True)
    except:
        pass
df.to_csv('processed_kakao.csv', index = False)

  0%|          | 0/936 [00:00<?, ?it/s]

In [6]:
df

,questions,answers
0,,밥 언제 ㄱ?
1,그렇다면 수금이 좋음,나는 다 갠춘
2,좋아,수저 ㄱ
3,우리 오랜만에 식당가는거지?,조와용
4,이모티콘,오늘 어디서 모여서 어디 가까
...,...,...
148,7월부터... 써칭할거야,ㅋㅋㅋㅋ웅웅
149,설렠ㅋㅋㅋㅋㅋ,이미 해본 사람이 옆에 있네
150,응,부럽넹 ㅋㅋㅋ 나도 해외에서 일해보고 싶은데
151,샌디에이고,오오


In [7]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 613 kB/s 
     |████████████████████████████████| 453 kB 25.5 MB/s 


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import json

from konlpy.tag import *

In [9]:
PAD = "<PADDING>"
STA = "<START>"
END = "<END>"
OOV = "<OOV>"

PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

max_sequences = 30
embedding_dim = 100
lstm_hidden_dim = 128

RE_FILTER = re.compile("[.,!?\"':;~()]")

In [62]:
chatbot_data = pd.read_csv('./processed_kakao.csv', encoding='utf-8')
questions_orig, answers_orig = list(chatbot_data['questions']), list(chatbot_data['answers'])

In [57]:
!ls

drive  processed_kakao.csv  sample_data


In [61]:
def pos_tag(sentences):
    tagger = Okt()
    sentences_pos = []
    for sentence in sentences:
        try:
            # 웃음 정규화
            sentence = re.sub('ㅋㅋㅋ+', 'ㅋㅋ', sentence)
            sentence = re.sub('ㅎㅎㅎ+', 'ㅎㅎ', sentence)
            # 특수기호 제거
            sentence = re.sub(RE_FILTER, "", sentence)
        except:
            sentence = "ㅇㅇ"
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence) 
    return sentences_pos

In [70]:
print(questions_orig)
for i in range(11):
    print('Q : {}'.format(questions_orig[i]))
    print('A : {}'.format(answers_orig[i]))
    print()

[nan, '그렇다면 수금이 좋음', '좋아', '우리 오랜만에 식당가는거지?', '이모티콘', '아무 생각이 없습니당', '괜찮지', '다들 어디써', '피자 나도 괜춘', '피자는 동물도 아닌걸...?', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '오늘??', '우리랑??', '스컴 짐은 할것 같던데', '나 탁구 짱못함', 'ㅋㅋㅋㅋ', '괜찮아', '나 양분순!', 'ㅋㅋㅋㅋㅋㅋ', '뒷덜미에 소름돋앗다', '6시?', '내가 4번인데 2번팀이 아직 미팅중이거든? 나까지 순서가 올까?', '우연이 편한대로 하자', '탁구채 있음?!', '탁구채 ㄱㄱㄱ', '나 유레카라 없는디', '케이아이', '4개나 있어야돼', '근데 안좋아 퀄리티', '두타입이잖아', '나두 양면이 좋음', '난도착', '나 도서관앞이야', '한판반?', 'ㄱㄱ', 'ㄱ', '궁금해서!', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '다들 잘지내니', '이상함', '즉석떡볶이 강추', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎ', '샌드위치 넘나 생존식품이네', '대박', '태평소 빨리가 점심때면 줄서야 할지도', '존맛이었다', 'ㅋㅋㅋㅋㅋㅋ 온대?', '벌써 벚꽃이라니', '벚꽃도', '목련 예쁘더라 ~', 'ㅋㅋㅋㅋ', '둘다**', '상표형 평일에 출장올일 없으려나', '갑자기 서울에 실험하러가야하는거아니면 다 괜찮은디', '애매할것 같아', '축하', '다음주면 언된다', '계속 바쁠듯 ㅜㅠ', '나빼고 고고', 'ㅋㅋㅋㅋㅋ', '아침에 더 잘 수 있음! (?)', '날 그냥 버려', 'ㅋㅋㅋ 그게 언제', '다음주 안돼', '15', '오 우현이 이사하는구나', '정신 읎다 읎어', 'ㅎ', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '응 다음에 초대할게', 'ㅋㅋㅋㅋㅋㅋㅋ 먼데 괜찮겠어?', '맞네', '요즘 너무 바글바글하더라', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '좋아', '좋아', '을 운전해서 간다고?', '저녁 같이드실분?', '솔아 셋이 ㄱ', 'ㅋㅋㅋㅋㅋㅋㅋㅋ', '완전바꼇네', '고고고고고고고

In [71]:
questions = pos_tag(questions_orig)
answers = pos_tag(answers_orig)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(11):
    print('Q : ' + questions[i])
    print('A : ' + answers[i])
    print()

Q : ㅇㅇ
A : 밥 언제 ㄱ

Q : 그렇다면 수금 이 좋음
A : 나 는 다 갠 춘

Q : 좋아
A : 수저 ㄱ

Q : 우리 오랜 만 에 식당가 는 거지
A : 조 와 용

Q : 이모티콘
A : 오늘 어디서 모 여서 어디 가까

Q : 아무 생각 이 없 습 니당
A : dddn

Q : 괜찮지
A : 사 다가 준우 연구실 에서 먹을까

Q : 다 들 어디 써
A : 나 랑 우현 누나 는 자 과동 이고 주누 는 유레카 관

Q : 피자 나도 괜춘
A : 내일 은 피자 돼서 일어나는 거야

Q : 피자 는 동물 도 아닌 걸
A : 혹시 아침 에 일어났는데 피자 되어있으면

Q : ㅋㅋ
A : 나 한 조각 만



In [72]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(questions)
sentences.extend(answers)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [73]:
# 단어 개수
len(words)

653

In [80]:
words[6]

'때'

In [15]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)} # 단어 -> 인덱스: 문장을 인덱스로 변환하여 모델 입력으로 사용
index_to_word = {index: word for index, word in enumerate(words)} # 인덱스 -> 단어: 모델의 예측 결과 인덱스를 문장으로 변환시 사용

In [16]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [85]:
x_encoder = convert_text_to_index(questions, word_to_index, ENCODER_INPUT) # 인코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answers, word_to_index, DECODER_INPUT) # 디코더 입력 인덱스 변환
y_decoder_index = convert_text_to_index(answers, word_to_index, DECODER_TARGET) # 디코더 목표 인덱스 변환

In [122]:
print(x_decoder.shape)

(153, 30)


In [88]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder_index), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder_index):
  for j, id in enumerate(sequence):
    one_hot_data[i, j, id] = 1
# 디코더 목표 설정
y_decoder = one_hot_data

In [109]:
# 첫 번째 디코더 목표 출력

print(y_decoder[0][9])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [108]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

Seq2Seq Keras LSTM 모델을 이해하기 위해서는 다음 링크 참조

https://wikidocs.net/24996
https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/


In [144]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]


#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [106]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [107]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [143]:
# 에폭 반복
for epoch in range(10):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도/손실 출력
    print('accuracy :', history.history['accuracy'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    input_encoder = x_encoder[13].reshape(1, x_encoder[13].shape[0])
    input_decoder = x_decoder[13].reshape(1, x_decoder[13].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
accuracy : 0.9281045794487
loss : 0.34361183643341064
ㅋㅋ 가 

Total Epoch : 2
accuracy : 0.9607843160629272
loss : 0.19232524931430817
ㅋㅋ 열었을걸 

Total Epoch : 3
accuracy : 0.9681917428970337
loss : 0.12484554201364517
난 열었을걸 

Total Epoch : 4
accuracy : 0.9699346423149109
loss : 0.09532027691602707
난 열었을걸 

Total Epoch : 5
accuracy : 0.9751634001731873
loss : 0.07751678675413132
27일 열었을걸 

Total Epoch : 6
accuracy : 0.9788671135902405
loss : 0.06426416337490082
서측 열었을걸 

Total Epoch : 7
accuracy : 0.9827886819839478
loss : 0.05183004215359688
서측 열었을걸 

Total Epoch : 8
accuracy : 0.9873638153076172
loss : 0.041693199425935745
서측 열었을걸 

Total Epoch : 9
accuracy : 0.9923747181892395
loss : 0.031628772616386414
서측 열었을걸 

Total Epoch : 10
accuracy : 0.9934640526771545
loss : 0.021888108924031258
서측 열었을걸 



In [29]:
input_decoder = x_decoder[13].reshape(1, x_decoder[13].shape[0])
results = model.predict([input_encoder, input_decoder])

In [114]:
indexs = np.argmax(results[0], 1)
sentence = convert_index_to_text(indexs, index_to_word)
print(sentence)

사진 열었을걸 


In [132]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [133]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [142]:
sen = '안녕'
input_seq = make_predict_input(sen)
print(input_seq)
output_seq = generate_text(input_seq)
print('[나한솔] {}'.format(output_seq))

[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[나한솔] 됐는지 됐는지 됐는지 다가 해주시겠지 다가 다 다 다 칼스버그 거야 양면 님 강도 강도 얼 가도 밑 저녁 후 포닥 포닥 애초 애초 아님 되지 14 아냐 설렠 유레카 


더 좋은 모델을 학습하기? 트랜스포머 이용

https://github.com/ukairia777/tensorflow-transformer

https://wikidocs.net/89786
